In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jcartus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Prepare Dataset 

In [2]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset

data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))

def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(index))

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_triu_norm, mu, std = AbstractDataset.normalize(s_triu)


s_train, p_train, s_test, p_test = split(s_triu_norm, p_triu, ind)

# Test Utilities 

## Model generation 

In [3]:
dim_triu = dim * (dim + 1) // 2

In [4]:
intializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)

def build_model(activation, structure, learning_rate):



    model = keras.Sequential()

    # input layer
    model.add(keras.layers.Dense(dim_triu, activation=activation, input_dim=dim_triu, kernel_initializer=intializer, bias_initializer='zeros'))

    # hidden
    for layer in structure:

        model.add(keras.layers.Dense(
                layer, 
                activation=activation, 
                kernel_initializer=intializer, 
                #bias_initializer='zeros',
                #kernel_regularizer=keras.regularizers.l2(1e-8)
        ))


    #output
    model.add(keras.layers.Dense(dim_triu))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='MSE', metrics=['mae', 'mse'])
    
    return model

## Training 

In [5]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_mean_squared_error", 
    min_delta=1e-7, 
    patience=50, 
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=10, 
    verbose=1, 
    mode='auto', 
    min_delta=1e-5, 
    cooldown=10, 
    min_lr=1e-10
)

In [6]:
n_tests = 10
n_resets = 4
epochs = 10000


def test_model(structure, activation, learning_rate=1e-4):
    error = []
    
    for i in range(n_tests):
        
        keras.backend.clear_session()
        model = build_model(activation, structure, learning_rate)
        
        for j in range(n_resets):
            keras.backend.set_value(model.optimizer.lr, learning_rate)
            
            history = model.fit(
                x = s_train,
                y = p_train,
                epochs=epochs,
                shuffle=True,
                validation_data=(s_test, p_test), 
                verbose=0, 
                callbacks=[
                    early_stopping, 
                    reduce_lr
                ]
            )
        
        error.append(np.min(history.history["val_mean_absolute_error"]))
        
        print(" - Test " + str(i+1) + ": " + str(error[-1]))
    
    return error

def run_layer_test_series(activation, structures, learning_rate):
    error = []
    for structure in structures:
        print("Layers: " + str(structure))
        error.append(test_model(structure, activation, learning_rate=learning_rate))
    return error

# Testing 

In [ ]:
structures = [
        [dim_triu, dim_triu],
        [dim_triu, dim_triu, dim_triu],
        [dim_triu + 100, dim_triu + 100],
        [dim_triu + 250, dim_triu + 150, dim_triu + 50],
        [dim_triu + 100, dim_triu + 200, dim_triu + 100],
        [dim_triu, dim_triu, dim_triu, dim_triu, dim_triu]
    ]
        
learning_rates = [1e-3, 1e-4, 1e-5]

In [ ]:
res = []
for lr in learning_rates:
    print("# lr" + str(lr))
    res.append(run_layer_test_series('elu', structures, lr))

# lr0.001
Layers: [351, 351]

Epoch 00051: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00077: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00102: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00121: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00140: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00159: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00179: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00198: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 00217: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 00236: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 00255: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 00274: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-0


Epoch 00122: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00141: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00160: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00179: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 00198: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 00217: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 00236: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 00255: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epoch 00274: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
Epoch 00278: early stopping
 - Test 2: 0.004277152657990728

Epoch 00050: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00086: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00113: ReduceLROnPlateau reducing lear


Epoch 00261: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epoch 00280: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.

Epoch 00299: ReduceLROnPlateau reducing learning rate to 6.103515914901436e-08.
Epoch 00304: early stopping

Epoch 00061: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00080: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00099: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00118: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00137: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00156: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00175: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00194: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 00213: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-0


Epoch 00091: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00110: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00129: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00148: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00167: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00186: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 00205: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 00224: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 00243: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 00262: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epoch 00281: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
Epoch 00281: early stopping

Epoch 00053: ReduceLROnPlateau reducing learning rate to 0.000500000023748725


Epoch 00195: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 00214: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 00233: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 00252: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 00271: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epoch 00290: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
Epoch 00299: early stopping

Epoch 00066: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00085: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00104: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00123: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00142: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00161: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-0

...


Epoch 00173: ReduceLROnPlateau reducing learning rate to 1.9531249506599124e-08.

Epoch 00192: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-09.

Epoch 00211: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-09.

Epoch 00230: ReduceLROnPlateau reducing learning rate to 2.4414061883248905e-09.
Epoch 00230: early stopping
 - Test 9: 0.011073939752445291

Epoch 00685: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00704: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 00723: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 00742: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.

Epoch 00761: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.

Epoch 00788: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-07.

Epoch 00807: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-08.

Epoch 00826: ReduceLROnPlateau reducing learning r

Epoch 00236: early stopping
 - Test 1: 0.01137479971538284

Epoch 00673: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00702: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 00725: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 00744: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.

Epoch 00763: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.

Epoch 00782: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-07.

Epoch 00801: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-08.

Epoch 00820: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-08.

Epoch 00839: ReduceLROnPlateau reducing learning rate to 1.9531249506599124e-08.

Epoch 00858: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-09.

Epoch 00877: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-09.

Epoch 00896: ReduceLROnPlateau reducing learning rat


Epoch 00747: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.

Epoch 00766: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-07.

Epoch 00785: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-08.

Epoch 00804: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-08.

Epoch 00823: ReduceLROnPlateau reducing learning rate to 1.9531249506599124e-08.

Epoch 00842: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-09.

Epoch 00861: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-09.
Epoch 00876: early stopping

Epoch 00020: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00049: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 00068: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 00087: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.

Epoch 00106: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.

Epo


Epoch 00830: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-08.

Epoch 00849: ReduceLROnPlateau reducing learning rate to 1.9531249506599124e-08.

Epoch 00868: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-09.

Epoch 00887: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-09.
Epoch 00902: early stopping

Epoch 00018: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00037: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 00057: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 00076: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.

Epoch 00095: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.

Epoch 00114: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-07.

Epoch 00133: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-08.

Epoch 00152: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-08.

Ep

Epoch 00927: early stopping

Epoch 00017: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00036: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 00057: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 00076: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.

Epoch 00095: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.

Epoch 00114: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-07.

Epoch 00133: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-08.

Epoch 00152: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-08.

Epoch 00171: ReduceLROnPlateau reducing learning rate to 1.9531249506599124e-08.

Epoch 00190: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-09.

Epoch 00209: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-09.
Epoch 00219: early stopping

Epoch 00020: ReduceLROnPlateau reducing learning rate t

In [ ]:
x = [ 
    s for s in range(len(structures)) for i in range(n_tests)
]

In [ ]:
def structure_string(structure):
    return "_".join(list(map(str,list(structure))))
structure_string(structures[0])

In [ ]:
for r, lr in zip(res, learning_rates):
    plt.scatter(x, np.array(r).flatten(), marker="x", label="$\eta$ = " + str(lr) )

#plt.xlabel("Number of Hidden Layers / 1")
plt.ylabel("Mean Absolute Error / 1")

plt.xticks(
    np.arange(len(structures)),
    [structure_string(structure) for structure in structures],
    rotation=-60
)

plt.ylim(0.0025, 0.015)

plt.legend()
plt.tight_layout()

plt.savefig("/home/jcartus/Repos/MastersThesis/SMatrixDescriptor/figures/HyperparameterStudy.pdf")

plt.show()

In [ ]:
(5e-3)**2